# Determining best combination of preprocessing techniques  

## Loading Libraries

In [68]:

#MÅ GÅ NØYE IGJENNOM Å SJEKKE HVA SOM FAKTISK BRUKES

%load_ext autoreload
%autoreload 2

#import random as rn
#import tensorflow as tf
#from keras import backend as K

# external imports
import numpy as np
#import pandas as pd
import seaborn as sb
#import gensim
#from gensim.scripts.glove2word2vec import glove2word2vec
#import pylab as pl
import matplotlib.pyplot as plt
import csv
#import scipy
#import os.path
import pickle

#import keras
#from keras.layers import *
#from keras.layers.core import *
#from keras import backend as K

import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.model_selection import ParameterGrid
#import time

# internal imports
import helpers as HL
import glove_module as GV
import neural_nets as NN
import tokenizing as TO


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import pretrained GloVe with gensim
one can use gensims word2vec functions to check similarity and other interesting functions https://radimrehurek.com/gensim/models/word2vec.html

## Create the word embeddings using the created gensim-.txt file.

In [2]:
#Pick one, the higher dimension, the better result and longer computational time. 
global_vectors=HL.get_global_vectors(200)

## Creating corpus:
In addition to the acutal corpus, some additional information is needed 

In [3]:
full_corpus, nr_pos_tweets, nr_neg_tweets, total_training_tweets=HL.get_corpus(test=True)

## Picking the neural net

In [4]:
neural_nets=[NN.basic_model_adam]

## Initializing variables to apply all preprocessing techniques:

In [ ]:
#Initializing vectors:

corpuses=[]
corpuses.append(full_corpus)


In [ ]:
#Defining names of corpuses: 
names=['original_corpus','SH_corpus','SHM_corpus','H_corpus','HK_corpus','PS_corpus','NS__corpus','OS_corpus','N_corpus','NM_corpus','ST_corpus','SP_corpus','E_corpus','SN_corpus','RS_corpus','EX_corpus','N-2_corpus','N-3_corpus','N-4_corpus']

In [ ]:
#Defining inputs to preprocessing function: 
inputs=[{'hashtag': True, 'segmentation_hash': True},
        {'hashtag':True,'segmentation_hash': True,'hashtag_mention':True},
        {'hearts':True},
        {'hugs_and_kisses':True},
        {'pos_smilies':True},
        {'neg_smilies':True},
        {'other_smilies':True},
        {'numbers':True},
        {'numbers':True,'number_mention':True},
        {'stemming':True},
        {'spelling':False},#Warning: When True, it takes app 149 minutes on test set. Recomended to always set to false 
        {'elongation':True},
        {'set_to_not':True},
        {'remove_signs':True},
        {'exclamation':True}]

## Applying all preprocessing techniques to the original corpus: 

In [ ]:

for input_ in inputs: 
        corpus=TO.preprocess_corpus(full_corpus, **input_)
        corpuses.append(corpus)
        

In [ ]:
ns=[2,3,4]
for n in ns: 
    corpus=HL.creating_n_grams_corpus(n,full_corpus)
    corpuses.append(corpus)

## Round 1: Testing all preprocessing techniques: 

In [ ]:
accuracies=[]
stds=[]

for corpus in corpuses: 
    model_score=GV.classify_with_neural_networks(neural_nets, global_vectors, corpus, total_training_tweets, nr_pos_tweets, epochs=100, n_folds=5)
    accuracies.append(model_score[0][0])
    stds.append(model_score[0][1])

In [ ]:
print(len(accuracies))

## Want to determine which preprocessing techniques that improved the accuracy, and keep them: 

In [ ]:
corpuses_1=[]
names_1=[]
stds_1=[]
acc_1=[]
print('The original corpus gave accuracy of: ',accuracies[0], 'std:', stds[0],'\n')
for i in range(1,len(accuracies)):
    if accuracies[i]>=accuracies[0]:
        corpuses_1.append(corpuses[i])
        names_1.append(names[i])
        stds_1.append(stds[i])
        acc_1.append(accuracies[i])
        print('IMPROVED:  ',names[i],', score:',accuracies[i],'std:',stds[i])
    else:
        print('Not better:',names[i],', score:',accuracies[i],'std:',stds[i])
        

In [ ]:
print(names_1)

"""
IMPROVED:   SH_corpus , score: 0.825355 std: 0.000791580697087
IMPROVED:   SHM_corpus , score: 0.825075 std: 0.00108085614214
Not better: H_corpus , score: 0.82204 std: 0.00155889383859
IMPROVED:   HK_corpus , score: 0.82262 std: 0.00126653069446
IMPROVED:   PS_corpus , score: 0.82248 std: 0.00136009190866
IMPROVED:   NS__corpus , score: 0.822385 std: 0.00153407952858
IMPROVED:   OS_corpus , score: 0.822515 std: 0.00137109445335
IMPROVED:   N_corpus , score: 0.822245 std: 0.00134651401775
IMPROVED:   NM_corpus , score: 0.822745 std: 0.00184183332579
Not better: ST_corpus , score: 0.812625 std: 0.000972239682383
IMPROVED:   SP_corpus , score: 0.822245 std: 0.00134651401775
IMPROVED:   E_corpus , score: 0.82334 std: 0.00197917912277
IMPROVED:   SN_corpus , score: 0.825595 std: 0.00118473625757
Not better: RS_corpus , score: 0.814735 std: 0.00153155150093
IMPROVED:   EX_corpus , score: 0.82229 std: 0.00142571034926
IMPROVED:   N-2_corpus , score: 0.823265 std: 0.00156073700539
Not better: N-3_corpus , score: 0.82214 std: 0.00145487112831
Not better: N-4_corpus , score: 0.816695 std: 0.00155212757208
"""

In [ ]:
# We do not keep SHM, as SH is better. 
# We do not keep N as NM is better 
# We do not keep SP due to time

# Round 2: Apply all techniques that contributed positively

In [5]:
input_={'hashtag': True, 'segmentation_hash': True,'hugs_and_kisses':True,'all_smilies':True,
        'numbers':True,'number_mention':True,'elongation':True, 'set_to_not':True,'exclamation':True}


In [6]:
all_prepr_corpus=TO.preprocess_corpus(full_corpus, **input_)

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Time in min before starting first for loop: 0.16607651313145955
Time in min after 1  tweets: 0.16622994740804037
Time in min after 25001  tweets: 0.4907379984855652
Time in min after 50001  tweets: 0.788178030649821
Time in min after 75001  tweets: 1.1086444497108459
Time in min after 100001  tweets: 1.430128562450409
Time in min after 125001  tweets: 1.7543797810872397
Time in min after 150001  tweets: 2.045871897538503
Time in min after 175001  tweets: 2.312680999437968
Time in min after 200001  tweets: 2.5763596653938294
Time in min total: 2.7269849300384523


In [82]:
corpuses=[]
corpuses.append(all_prepr_corpus)
ns=[2]
for n in ns: 
    corpus=HL.creating_n_grams_corpus(n,all_prepr_corpus)
    #corpuses.append(corpus)

In [8]:
accuracies2=[]
stds2=[]

for corpus in corpuses: 
    model_score=GV.classify_with_neural_networks(neural_nets, global_vectors, corpus, total_training_tweets, nr_pos_tweets, epochs=100, n_folds=5)
    accuracies2.append(model_score[0][0])
    stds2.append(model_score[0][1])

tweets processed: 0  of total number of tweets: 200000
tweets processed: 50000  of total number of tweets: 200000
tweets processed: 100000  of total number of tweets: 200000
tweets processed: 150000  of total number of tweets: 200000
Train on 160000 samples, validate on 40000 samples
Epoch 1/100
160000/160000 [==============================] - 3s 17us/step - loss: 0.4509 - acc: 0.7776 - val_loss: 0.4035 - val_acc: 0.8114
Epoch 2/100
160000/160000 [==============================] - 3s 17us/step - loss: 0.3916 - acc: 0.8176 - val_loss: 0.3892 - val_acc: 0.8194
Epoch 3/100
160000/160000 [==============================] - 3s 16us/step - loss: 0.3795 - acc: 0.8235 - val_loss: 0.3828 - val_acc: 0.8228
Epoch 4/100
160000/160000 [==============================] - 2s 15us/step - loss: 0.3708 - acc: 0.8287 - val_loss: 0.3808 - val_acc: 0.8242
Epoch 5/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3645 - acc: 0.8320 - val_loss: 0.3765 - val_acc: 0.8271
Epoch 6/100
160

Epoch 00012: early stopping
40000/40000 [==============================] - 1s 37us/step
Train on 160000 samples, validate on 40000 samples
Epoch 1/100
160000/160000 [==============================] - 4s 22us/step - loss: 0.4520 - acc: 0.7769 - val_loss: 0.4013 - val_acc: 0.8114
Epoch 2/100
160000/160000 [==============================] - 2s 13us/step - loss: 0.3924 - acc: 0.8172 - val_loss: 0.3871 - val_acc: 0.8197
Epoch 3/100
160000/160000 [==============================] - 2s 13us/step - loss: 0.3796 - acc: 0.8242 - val_loss: 0.3805 - val_acc: 0.8226
Epoch 4/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3714 - acc: 0.8284 - val_loss: 0.3775 - val_acc: 0.8250
Epoch 5/100
160000/160000 [==============================] - 2s 14us/step - loss: 0.3656 - acc: 0.8322 - val_loss: 0.3740 - val_acc: 0.8276
Epoch 6/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3594 - acc: 0.8351 - val_loss: 0.3718 - val_acc: 0.8278
Epoch 7/100
160000/16

Epoch 4/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3711 - acc: 0.8287 - val_loss: 0.3789 - val_acc: 0.8222
Epoch 5/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3649 - acc: 0.8323 - val_loss: 0.3774 - val_acc: 0.8241
Epoch 6/100
160000/160000 [==============================] - 2s 13us/step - loss: 0.3599 - acc: 0.8356 - val_loss: 0.3750 - val_acc: 0.8244
Epoch 7/100
160000/160000 [==============================] - 2s 14us/step - loss: 0.3552 - acc: 0.8377 - val_loss: 0.3755 - val_acc: 0.8246
Epoch 8/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3506 - acc: 0.8398 - val_loss: 0.3735 - val_acc: 0.8270
Epoch 9/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3471 - acc: 0.8421 - val_loss: 0.3731 - val_acc: 0.8266
Epoch 10/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3433 - acc: 0.8443 - val_loss: 0.3743 - val_acc: 0.8266
Epoch 11/100
160000

Epoch 14/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3291 - acc: 0.8520 - val_loss: 0.3764 - val_acc: 0.8269
Epoch 00014: early stopping
40000/40000 [==============================] - 3s 76us/step
Train on 160000 samples, validate on 40000 samples
Epoch 1/100
160000/160000 [==============================] - 3s 19us/step - loss: 0.4496 - acc: 0.7767 - val_loss: 0.3987 - val_acc: 0.8145
Epoch 2/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3899 - acc: 0.8176 - val_loss: 0.3855 - val_acc: 0.8217
Epoch 3/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3777 - acc: 0.8243 - val_loss: 0.3794 - val_acc: 0.8245
Epoch 4/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3696 - acc: 0.8289 - val_loss: 0.3754 - val_acc: 0.8274
Epoch 5/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3627 - acc: 0.8327 - val_loss: 0.3762 - val_acc: 0.8256
Epoch 6/100
160000/1

Epoch 4/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3696 - acc: 0.8292 - val_loss: 0.3765 - val_acc: 0.8247
Epoch 5/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3633 - acc: 0.8328 - val_loss: 0.3725 - val_acc: 0.8268
Epoch 6/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3573 - acc: 0.8359 - val_loss: 0.3713 - val_acc: 0.8274
Epoch 7/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3530 - acc: 0.8387 - val_loss: 0.3696 - val_acc: 0.8295
Epoch 8/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3477 - acc: 0.8418 - val_loss: 0.3690 - val_acc: 0.8303
Epoch 9/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3444 - acc: 0.8433 - val_loss: 0.3708 - val_acc: 0.8296
Epoch 10/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3406 - acc: 0.8457 - val_loss: 0.3714 - val_acc: 0.8282
Epoch 11/100
160000

Epoch 6/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3584 - acc: 0.8347 - val_loss: 0.3753 - val_acc: 0.8269
Epoch 7/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3537 - acc: 0.8370 - val_loss: 0.3730 - val_acc: 0.8285
Epoch 8/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3493 - acc: 0.8395 - val_loss: 0.3724 - val_acc: 0.8279
Epoch 9/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3458 - acc: 0.8421 - val_loss: 0.3724 - val_acc: 0.8284
Epoch 10/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3420 - acc: 0.8437 - val_loss: 0.3720 - val_acc: 0.8291
Epoch 11/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3388 - acc: 0.8457 - val_loss: 0.3714 - val_acc: 0.8287
Epoch 12/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3359 - acc: 0.8468 - val_loss: 0.3716 - val_acc: 0.8290
Epoch 13/100
1600

160000/160000 [==============================] - 2s 11us/step - loss: 0.3500 - acc: 0.8398 - val_loss: 0.3697 - val_acc: 0.8275
Epoch 9/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3459 - acc: 0.8426 - val_loss: 0.3688 - val_acc: 0.8296
Epoch 10/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3418 - acc: 0.8445 - val_loss: 0.3693 - val_acc: 0.8310
Epoch 11/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3387 - acc: 0.8468 - val_loss: 0.3701 - val_acc: 0.8282
Epoch 12/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3360 - acc: 0.8479 - val_loss: 0.3710 - val_acc: 0.8278
Epoch 00012: early stopping
40000/40000 [==============================] - 2s 40us/step
Train on 160000 samples, validate on 40000 samples
Epoch 1/100
160000/160000 [==============================] - 4s 25us/step - loss: 0.4484 - acc: 0.7775 - val_loss: 0.4014 - val_acc: 0.8097
Epoch 2/100
160000/160000 [===

Epoch 11/100
160000/160000 [==============================] - 2s 10us/step - loss: 0.3391 - acc: 0.8459 - val_loss: 0.3734 - val_acc: 0.8257
Epoch 12/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3359 - acc: 0.8480 - val_loss: 0.3707 - val_acc: 0.8278
Epoch 13/100
160000/160000 [==============================] - 2s 10us/step - loss: 0.3334 - acc: 0.8495 - val_loss: 0.3728 - val_acc: 0.8270
Epoch 14/100
160000/160000 [==============================] - 2s 10us/step - loss: 0.3306 - acc: 0.8505 - val_loss: 0.3721 - val_acc: 0.8263
Epoch 15/100
160000/160000 [==============================] - 2s 10us/step - loss: 0.3284 - acc: 0.8518 - val_loss: 0.3728 - val_acc: 0.8268
Epoch 00015: early stopping
40000/40000 [==============================] - 2s 43us/step
Model:  basic_model_adam
[0.82869999999999999, 0.828125, 0.82962499999999995, 0.83007500000000001, 0.82784999999999997]
0.83% (+/- 0.00%)
Negative sentiment: 80.48%  Positive sentiment: 85.30%
Percentage of 

160000/160000 [==============================] - 2s 11us/step - loss: 0.3708 - acc: 0.8267 - val_loss: 0.3849 - val_acc: 0.8190
Epoch 6/100
160000/160000 [==============================] - 2s 13us/step - loss: 0.3657 - acc: 0.8293 - val_loss: 0.3816 - val_acc: 0.8205
Epoch 7/100
160000/160000 [==============================] - 2s 13us/step - loss: 0.3612 - acc: 0.8317 - val_loss: 0.3806 - val_acc: 0.8220
Epoch 8/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3567 - acc: 0.8349 - val_loss: 0.3813 - val_acc: 0.8215
Epoch 9/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3534 - acc: 0.8361 - val_loss: 0.3807 - val_acc: 0.8224
Epoch 10/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3499 - acc: 0.8388 - val_loss: 0.3829 - val_acc: 0.8207
Epoch 00010: early stopping
40000/40000 [==============================] - 3s 73us/step
Train on 160000 samples, validate on 40000 samples
Epoch 1/100
160000/160000 [=====

Epoch 9/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3543 - acc: 0.8364 - val_loss: 0.3789 - val_acc: 0.8231
Epoch 10/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3506 - acc: 0.8390 - val_loss: 0.3781 - val_acc: 0.8235
Epoch 11/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3469 - acc: 0.8409 - val_loss: 0.3771 - val_acc: 0.8240
Epoch 00011: early stopping
40000/40000 [==============================] - 2s 51us/step
Train on 160000 samples, validate on 40000 samples
Epoch 1/100
160000/160000 [==============================] - 5s 30us/step - loss: 0.4540 - acc: 0.7718 - val_loss: 0.4073 - val_acc: 0.8043
Epoch 2/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3959 - acc: 0.8127 - val_loss: 0.3958 - val_acc: 0.8135
Epoch 3/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3849 - acc: 0.8186 - val_loss: 0.3892 - val_acc: 0.8146
Epoch 4/100
160000/

In [9]:
for i in range(len(corpuses)):
    print('Accuracy:',  accuracies2[i], 'std:',stds2[i],'\n' )

Accuracy: 0.828935 std: 0.000962626615049 

Accuracy: 0.829455 std: 0.00127822142057 

Accuracy: 0.828875 std: 0.00085396135744 

Accuracy: 0.82377 std: 0.000920380356157 



In [10]:
best_corpus=corpuses[1]

### Dynamic stopword list

In [ ]:
mindfs= np.linspace(0.0000005, 0.00001, num=5)
maxdfs=0.8 #np.linspace(0.3, 0.8, num=1)
accuracies_stop=[]
stds_stop=[]
stop_lens=[]
vocabs=[]


#for max_ in maxdfs:
for min_ in mindfs: 
    stopwords, vocab= TO.get_dynamic_stopwords(corpus, MinDf=min_, MaxDf=max_,sublinearTF=True,useIDF=False)
    stopword_corpus=TO.remove_stopwords(corpus, stopwords)
    model_score=GV.classify_with_neural_networks(neural_nets, global_vectors, stopword_corpus, total_training_tweets, nr_pos_tweets, epochs=100, n_folds=5)
    #accuracies_stop.append(model_score[0][0])
    #stds_stop.append(model_score[0][1])
    stop_lens.append(len(stopwords))
    vocabs.append(vocab)

    #print('This is min',min_, 'this is max', max_, 'this is len stopwords', len(stopwords), 'this is acc: ', model_score[0][0],'\n')


Size of Vocabulary: 784494
tweets processed: 0  of total number of tweets: 200000
tweets processed: 50000  of total number of tweets: 200000
tweets processed: 100000  of total number of tweets: 200000
tweets processed: 150000  of total number of tweets: 200000
Train on 160000 samples, validate on 40000 samples
Epoch 1/100
160000/160000 [==============================] - 61s 384us/step - loss: 0.4526 - acc: 0.7761 - val_loss: 0.4048 - val_acc: 0.8100
Epoch 2/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3932 - acc: 0.8165 - val_loss: 0.3901 - val_acc: 0.8185
Epoch 3/100
160000/160000 [==============================] - 2s 13us/step - loss: 0.3806 - acc: 0.8229 - val_loss: 0.3834 - val_acc: 0.8226
Epoch 4/100
160000/160000 [==============================] - 2s 15us/step - loss: 0.3718 - acc: 0.8284 - val_loss: 0.3812 - val_acc: 0.8244
Epoch 5/100
160000/160000 [==============================] - 2s 13us/step - loss: 0.3652 - acc: 0.8325 - val_loss: 0.3767 - va

Epoch 00012: early stopping
40000/40000 [==============================] - 4s 90us/step
Train on 160000 samples, validate on 40000 samples
Epoch 1/100
160000/160000 [==============================] - 8s 50us/step - loss: 0.4535 - acc: 0.7758 - val_loss: 0.4031 - val_acc: 0.8109
Epoch 2/100
160000/160000 [==============================] - 2s 13us/step - loss: 0.3938 - acc: 0.8161 - val_loss: 0.3885 - val_acc: 0.8206
Epoch 3/100
160000/160000 [==============================] - 2s 15us/step - loss: 0.3808 - acc: 0.8237 - val_loss: 0.3817 - val_acc: 0.8223
Epoch 4/100
160000/160000 [==============================] - 2s 13us/step - loss: 0.3725 - acc: 0.8279 - val_loss: 0.3784 - val_acc: 0.8248
Epoch 5/100
160000/160000 [==============================] - 2s 15us/step - loss: 0.3666 - acc: 0.8320 - val_loss: 0.3752 - val_acc: 0.8277
Epoch 6/100
160000/160000 [==============================] - 2s 15us/step - loss: 0.3604 - acc: 0.8350 - val_loss: 0.3730 - val_acc: 0.8283
Epoch 7/100
160000/16

160000/160000 [==============================] - 2s 11us/step - loss: 0.3806 - acc: 0.8236 - val_loss: 0.3845 - val_acc: 0.8203
Epoch 4/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3719 - acc: 0.8282 - val_loss: 0.3801 - val_acc: 0.8220
Epoch 5/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3656 - acc: 0.8322 - val_loss: 0.3788 - val_acc: 0.8234
Epoch 6/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3607 - acc: 0.8355 - val_loss: 0.3762 - val_acc: 0.8246
Epoch 7/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3559 - acc: 0.8377 - val_loss: 0.3770 - val_acc: 0.8250
Epoch 8/100
160000/160000 [==============================] - 2s 11us/step - loss: 0.3513 - acc: 0.8401 - val_loss: 0.3746 - val_acc: 0.8258
Epoch 9/100
160000/160000 [==============================] - 2s 12us/step - loss: 0.3478 - acc: 0.8427 - val_loss: 0.3744 - val_acc: 0.8262
Epoch 10/100
160000/160000 [====

In [ ]:
for i in range(len(stds_stop)):
    print('Accuracy:',  accuracies_stop[i], 'std:',stds_stop[i],'stopwords:',stop_lens[i],'\n' )

# Testing "best preprocessing" with full dataset: 

Som før for å lage en keggle! 

In [ ]:
final_corpus=TO.preprocess_corpus(full_corpus, segmentation_hash=True, hashtag=True, hashtag_mention=True, set_to_not=True,elongation=True)


In [ ]:
model_score=GV.classify_with_neural_networks(neural_nets, global_vectors, final_corpus, total_training_tweets, nr_pos_tweets, epochs=6, n_folds=3)

# Making kaggle submission: 

In [ ]:
kaggle_name="keggle_glove_test_adam.csv"
#final_corpus=n_grams_corpus

pred= GV.get_prediction(NN.basic_model_adam, global_vectors, full_corpus, total_training_tweets, nr_pos_tweets,kaggle_name, epochs=100)

In [ ]:
print(sum(pred))

In [ ]:
import enchant
d = enchant.Dict("en_US")